In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-games/players.csv
/kaggle/input/nba-games/teams.csv
/kaggle/input/nba-games/ranking.csv
/kaggle/input/nba-games/games_details.csv
/kaggle/input/nba-games/games.csv


## Initial analysis of relevant data

Before we begin to code our ML model we must first look into what data is not only relevant to what it is we are trying to accomplish but if the data is related to the desired outcome. In this particular case we can define our desired outcome as;

    -Predict the outcome of NBA games

Though vague it is important to have an understanding of the ultimate goal. With that in mind we must determine what is considered relevant to being able to make these predictions. For the purposes of this analysis we will look at what data appears to relate to winning a game to have a more concrete idea of what features to include in the model.

